# Workflow

In [ ]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import sys

sys.path.append('../hydra_tod/')

from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u
from utils import Leg_poly_proj
from flicker_model import sim_noise, flicker_cov
from simulation import * 
import pickle


## Simulation

In [4]:
# Create the simulation instance
tod_sim = TODSimulation()

# Print simulation summary
print("\nSimulation Summary:")
for key, value in tod_sim.get_simulation_summary().items():
    print(f"{key}: {value}")

# Save the class object of TODSimulation
# This can be done using pickle or any other serialization method


Number of pixels: 375
Gain parameters: [6.31194264 0.42038942 0.264222   0.05578821]

Simulation Summary:
nside: 64
n_pixels: 375
n_time_samples: 2860
frequency_MHz: 750
beam_FWHM_deg: 1.1
elevation_deg: 41.5
azimuth_range_deg: (-60.3, -42.3)
gain_params: [6.31194264 0.42038942 0.264222   0.05578821]
noise_knee_freq_Hz: 1.3348390111267891e-05
noise_spectral_index: 2.0
T_ndiode_K: 15.0
receiver_params_K: [12.6  0.5  0.5  0.5]

Simulation Summary:
nside: 64
n_pixels: 375
n_time_samples: 2860
frequency_MHz: 750
beam_FWHM_deg: 1.1
elevation_deg: 41.5
azimuth_range_deg: (-60.3, -42.3)
gain_params: [6.31194264 0.42038942 0.264222   0.05578821]
noise_knee_freq_Hz: 1.3348390111267891e-05
noise_spectral_index: 2.0
T_ndiode_K: 15.0
receiver_params_K: [12.6  0.5  0.5  0.5]


In [ ]:
with open('tod_simulation_single.pkl', 'wb') as f:
    pickle.dump(tod_sim, f)

In [2]:

# Create the multi-TOD simulation instance
multi_tod_sim = MultiTODSimulation()

# Print simulation summary
print("\nMulti-TOD Simulation Summary:")
summary = multi_tod_sim.get_simulation_summary()
for key, value in summary.items():
    if isinstance(value, dict):
        print(f"{key}:")
        for subkey, subvalue in value.items():
            print(f"  {subkey}: {subvalue}")
    else:
        print(f"{key}: {value}")

Number of pixels: 473

Multi-TOD Simulation Summary:
nside: 64
n_pixels: 473
n_time_samples: 2860
frequency_MHz: 750
beam_FWHM_deg: 1.1
setting_scan:
  elevation_deg: 41.5
  azimuth_range_deg: (-60.3, -42.3)
  gain_params: [6.31194264 0.42038942 0.264222   0.05578821]
rising_scan:
  elevation_deg: 40.5
  azimuth_range_deg: (43.7, 61.7)
  gain_params: [6.84507868 0.14156859 0.7441104  0.77863955]
noise_knee_freq_Hz: 1.3348390111267891e-05
noise_spectral_index: 2.0
T_ndiode_K: 15.0
receiver_params_K: [12.6  0.5  0.5  0.5]
calibration_pixels:
  single_cal_index: [149]
  five_cal_indices: [np.int64(394), np.int64(344), np.int64(281), np.int64(254), np.int64(176)]


In [3]:
# Save the simulation data to a pickle file
with open('multi_tod_simulation_data.pkl', 'wb') as f:
    pickle.dump(multi_tod_sim, f)